# LangChain Agent实战：从基础工具链到智能诊断系统

## 一、LangChain Agent简介

LangChain是一个强大的框架，它允许我们将大语言模型(LLM)与外部工具和数据源结合起来，构建更智能、更实用的应用。其中，Agent(智能代理)是LangChain中最令人兴奋的功能之一，它能够自主决策使用哪些工具来完成任务，实现类似"思考-行动-观察"的工作流程。
本文将带您从零开始，通过实战案例了解LangChain Agent的开发与应用，从简单的工具链到复杂的智能诊断系统，逐步提升您的Agent开发能力。

## 二、环境准备

首先，我们需要安装必要的库并配置环境变量：

In [19]:
%pip install --upgrade jieba

Note: you may need to restart the kernel to use updated packages.


In [1]:
# 安装必要的库
# %pip install langchain langchain-community python-dotenv

# 引入基础库
import os
from dotenv import load_dotenv, find_dotenv

# 加载环境变量
_ = load_dotenv(find_dotenv())

# 设置API密钥
DASHSCOPE_API_KEY = os.getenv("DASHSCOPE_API_KEY")

## 三、基础工具链开发

### 3.1 自定义工具开发

LangChain的Agent需要工具(Tool)来执行具体任务。下面我们创建一个简单的文本分析工具：

In [23]:
from langchain.agents import Tool
from langchain_community.llms import Tongyi
import jieba

# 自定义工具：文本分析工具
class TextAnalysisTool:
    """文本分析工具，用于分析文本内容"""
    
    def __init__(self):
        self.name = "文本分析"
        self.description = "分析文本内容，提取字数、字符数和情感倾向"
    
    def run(self, text: str) -> str:
        """分析文本内容"""
        # 计算词数和字符数
        # word_count = len([word for word in text.split() if word.strip()]) # 计算词数 适合英文
        word_count = len([word for word in list(jieba.cut(text)) if word.strip()]) # 计算词数 适合中文
        char_count = len(text) # 计算字符数
        
        # 简单的情感分析
        positive_words = ["好", "优秀", "喜欢", "快乐", "成功", "美好"]
        negative_words = ["差", "糟糕", "讨厌", "悲伤", "失败", "痛苦"]
        
        positive_count = sum(1 for word in positive_words if word in text)
        negative_count = sum(1 for word in negative_words if word in text)
        
        sentiment = "积极" if positive_count > negative_count else "消极" if negative_count > positive_count else "中性"
        
        return f"文本分析结果:\n- 词数: {word_count}\n- 字符数: {char_count}\n- 情感倾向: {sentiment}"

### 3.2 构建传统Agent工具链

下面我们将创建一个完整的Agent，它可以根据需要自动选择合适的工具：

In [24]:
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

# ConversationBufferMemory:作用是保存对话历史，以便Agent可以参考之前的对话内容。
# ConversationBufferWindowMemory:作用是保存最近的对话历史，以便Agent可以参考最近的对话内容。
# ConversationSummaryMemory:作用是保存对话历史，以便Agent可以参考之前的对话内容。
# ConversationEntityMemory:作用是保存对话历史，以便Agent可以参考之前的对话内容。

def create_tool_chain():
    """创建工具链"""
    # 创建工具实例
    text_analysis = TextAnalysisTool()
    
    # 将工具包装成LangChain Tool对象
    tools = [
        Tool(
            name=text_analysis.name,
            func=text_analysis.run,
            description=text_analysis.description
        )
    ]
    
    # 初始化语言模型
    llm = Tongyi(model_name="qwen-turbo", dashscope_api_key=DASHSCOPE_API_KEY)
    
    # 创建提示模板
    prompt = PromptTemplate.from_template(
        """你是一个有用的AI助手，可以使用以下工具:
{tools}
可用工具名称: {tool_names}

使用以下格式:
问题: 你需要回答的问题
思考: 你应该始终思考要做什么
行动: 要使用的工具名称，必须是 [{tool_names}] 中的一个
行动输入: 工具的输入
观察: 工具的结果
... (这个思考/行动/行动输入/观察可以重复 N 次)
思考: 我现在已经有了最终答案
回答: 对原始问题的最终回答

开始!
问题: {input}
思考: {agent_scratchpad}"""
    )
    
    # 创建Agent
    agent = create_react_agent(llm, tools, prompt)
    
    # 创建Agent执行器
    agent_executor = AgentExecutor.from_agent_and_tools(
        agent=agent,
        tools=tools,
        memory=ConversationBufferMemory(memory_key="chat_history"),
        verbose=True,
        handle_parsing_errors=False
    )
    
    return agent_executor

# 使用工具链处理任务
def process_task(task_description):
    agent_executor = create_tool_chain()
    response = agent_executor.invoke({"input": task_description})
    return response["output"]

### 3.3 使用LCEL简化工具链

LangChain Expression Language (LCEL)是LangChain的新一代构建方式，它使得创建复杂的工具链变得更加直观和简洁：

In [25]:
from langchain_core.runnables import RunnableLambda

# 工具实例
text_analysis = TextAnalysisTool()

# LCEL风格的工具链
tools = {
    "文本分析": RunnableLambda(lambda x: text_analysis.run(x["text"]))
}

# LCEL任务链
def lcel_task_chain(task_type, params):
    """LCEL风格的任务链调度"""
    if task_type not in tools:
        return "不支持的工具类型"
    return tools[task_type].invoke(params)

# 示例用法
result = lcel_task_chain("文本分析", {"text": "这个产品非常好用，我很喜欢它的设计，使用体验非常棒！"})
print(result)

文本分析结果:
- 词数: 17
- 字符数: 26
- 情感倾向: 积极


## 四、高级Agent应用：网络故障诊断系统

接下来，我们将开发一个更复杂的Agent应用：网络故障诊断系统。它能够根据用户描述的网络问题，自动选择合适的工具进行诊断。

### 4.1 创建专业诊断工具

首先，我们创建几个专业的网络诊断工具：

In [26]:
import re
from typing import Optional

# Ping工具
class PingTool:
    """执行Ping操作以检查网络连通性"""
    def __init__(self):
        self.name = "网络连通性检查 (Ping)"
        self.description = "检查本机到指定主机名或IP地址的网络连通性。输入应该是目标主机名或IP地址。"
    
    def run(self, target: str) -> str:
        print(f"--- 执行Ping: {target} ---")
        # 模拟Ping结果
        if "unreachable" in target or target == "192.168.1.254":
            return f"Ping {target} 失败：请求超时。"
        else:
            import random
            delay = random.randint(5, 50)
            return f"Ping {target} 成功：延迟 {delay}ms。"

# DNS查询工具
class DNSTool:
    """执行DNS查询以解析主机名"""
    def __init__(self):
        self.name = "DNS解析查询"
        self.description = "解析给定的主机名，获取其对应的IP地址。输入应该是要解析的主机名。"
    
    def run(self, hostname: str) -> str:
        print(f"--- DNS查询: {hostname} ---")
        # 模拟DNS解析
        if hostname == "unknown.domain.xyz":
            return f"DNS解析 {hostname} 失败：找不到主机。"
        elif hostname == "127.0.0.1" or re.match(r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", hostname):
            return f"输入 '{hostname}' 已经是IP地址，无需DNS解析。"
        else:
            return f"DNS解析 {hostname} 成功：IP地址是 10.0.0.5"

# 网络接口检查工具
class InterfaceCheckTool:
    """检查本地网络接口的状态"""
    def __init__(self):
        self.name = "本地网络接口检查"
        self.description = "检查本机网络接口的状态（如IP地址、是否启用）。"
    
    def run(self, interface_name: Optional[str] = None) -> str:
        interface = interface_name or "默认接口"
        print(f"--- 检查接口状态: {interface} ---")
        if interface_name and "eth1" in interface_name.lower():
            return f"接口 '{interface_name}' 状态：关闭"
        else:
            return f"接口 'Ethernet'/'Wi-Fi' 状态：启用, IP地址: 192.168.1.50"

### 4.2 构建网络诊断Agent

接下来，我们将这些工具组合成一个完整的网络诊断Agent：

In [27]:
from langchain.agents import Tool, AgentExecutor, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory

def create_network_diagnosis_chain():
    """创建网络故障诊断的Agent执行器"""
    # 初始化工具
    ping_tool = PingTool()
    dns_tool = DNSTool()
    interface_tool = InterfaceCheckTool()
    
    # 将工具类包装成LangChain Tool对象
    tools = [
        Tool(
            name=ping_tool.name,
            func=ping_tool.run,
            description=ping_tool.description
        ),
        Tool(
            name=dns_tool.name,
            func=dns_tool.run,
            description=dns_tool.description
        ),
        Tool(
            name=interface_tool.name,
            func=interface_tool.run,
            description=interface_tool.description
        )
    ]
    
    # 初始化语言模型
    llm = Tongyi(model_name="qwen-turbo", dashscope_api_key=DASHSCOPE_API_KEY)
    
    # 创建Agent
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=False,
        memory=ConversationBufferMemory(memory_key="chat_history", input_key="input", output_key="output"),
        max_iterations=10
    )
    
    return agent

# 诊断网络问题的函数
def diagnose_network_issue(issue_description: str):
    """使用网络诊断Agent处理用户报告的网络问题"""
    try:
        print(f"\n--- 开始诊断任务 ---")
        print(f"用户问题: {issue_description}")
        agent = create_network_diagnosis_chain()
        response = agent.invoke({"input": issue_description})
        return response.get("output", "未能获取诊断结果。")
    except Exception as e:
        print(f"处理诊断任务时发生错误: {str(e)}")
        return f"处理诊断任务时出错: {str(e)}"

### 4.3 使用诊断Agent解决实际问题

最后，我们使用这个Agent来诊断一些常见的网络问题：

In [28]:
# 示例1: 无法访问特定网站
task1 = "我无法访问www.example.com，浏览器显示连接超时。"
result1 = diagnose_network_issue(task1)
print(f"最终诊断结果: {result1}")

# 示例2: 内部服务访问失败
task2 = "连接到内部数据库服务器(internal.service.local)失败，提示'connection refused'。"
result2 = diagnose_network_issue(task2)
print(f"最终诊断结果: {result2}")


--- 开始诊断任务 ---
用户问题: 我无法访问www.example.com，浏览器显示连接超时。


> Entering new AgentExecutor chain...


C:\Users\Cheng\AppData\Local\Temp\ipykernel_14348\1805082966.py:40: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory=ConversationBufferMemory(memory_key="chat_history", input_key="input", output_key="output"),
C:\Users\Cheng\AppData\Local\Temp\ipykernel_14348\1805082966.py:34: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`

我需要检查本机到www.example.com的网络连通性。
Action: 网络连通性检查 (Ping)
Action Input: www.example.com--- 执行Ping: www.example.com ---

Observation: Ping www.example.com 成功：延迟 26ms。
Thought:网络连通性检查显示可以成功访问 www.example.com，因此问题可能出在其他地方，例如DNS解析或本地网络配置。
Action: DNS解析查询
Action Input: www.example.com--- DNS查询: www.example.com ---

Observation: DNS解析 www.example.com 成功：IP地址是 10.0.0.5
Thought:DNS解析查询显示www.example.com可以成功解析为IP地址10.0.0.5，因此DNS解析没有问题。现在我需要检查本地网络接口的状态，以确保本机网络配置正确。
Action: 本地网络接口检查  
Action Input: None  --- 检查接口状态: None ---

Observation: 接口 'Ethernet'/'Wi-Fi' 状态：启用, IP地址: 192.168.1.50
Thought:本地网络接口检查显示网络接口已启用，且IP地址配置正确。由于网络连通性、DNS解析和本地网络配置均无问题，问题可能出在其他方面，例如浏览器缓存、防火墙设置或目标网站的特定访问限制。

Final Answer: 经过检查，网络连通性、DNS解析和本地网络配置均正常。建议尝试以下方法解决连接超时问题：
1. 清除浏览器缓存并刷新页面。
2. 暂时关闭防火墙或检查安全软件设置。
3. 尝试使用其他浏览器或设备访问www.example.com。
4. 如果问题仍然存在，请联系网络管理员或互联网服务提供商以获取进一步支持。

> Finished chain.
最终诊断结果: 经过检查，网络连通性、DNS解析和本地网络配置均正常。建议尝试以下方法解决连接超时问题：
1. 清除浏览器缓存并刷新页面。
2. 暂时关闭防火墙或检查安全软件设置。
3. 尝试使用其他浏览器或设备访问www.example.com。
4. 如果问题仍然

## 五、构建自己的智能助手应用

基于上述案例，现在让我们开发一个更实用的应用场景：智能客服助手。这个助手可以处理客户咨询、产品推荐和故障诊断。

### 5.1 创建智能客服所需工具

In [29]:
# 产品查询工具
class ProductLookupTool:
    """查询产品信息的工具"""
    def __init__(self):
        self.name = "产品信息查询"
        self.description = "查询产品的详细信息，包括价格、规格、可用性等。输入应该是产品名称或编号。"
        # 模拟产品数据库
        self.products = {
            "智能手表A1": {"价格": "¥1299", "库存": "有货", "特点": "心率监测、GPS定位、续航7天"},
            "智能音箱S2": {"价格": "¥699", "库存": "有货", "特点": "高保真音质、智能语音助手、多房间联动"},
            "智能门锁M3": {"价格": "¥2499", "库存": "缺货", "特点": "指纹解锁、人脸识别、远程控制"}
        }
    
    def run(self, product_name: str) -> str:
        print(f"--- 查询产品: {product_name} ---")
        if product_name in self.products:
            product = self.products[product_name]
            return f"产品名称: {product_name}\n价格: {product['价格']}\n库存状态: {product['库存']}\n特点: {product['特点']}"
        else:
            return f"未找到产品 '{product_name}' 的信息。请尝试其他产品名称。"

# 订单查询工具
class OrderStatusTool:
    """查询订单状态的工具"""
    def __init__(self):
        self.name = "订单状态查询"
        self.description = "查询订单的当前状态和物流信息。输入应该是订单号。"
        # 模拟订单数据
        self.orders = {
            "ORD2023001": {"状态": "已发货", "物流": "顺丰快递 SF1234567890", "预计送达": "11月15日"},
            "ORD2023002": {"状态": "待发货", "物流": "尚未发出", "预计送达": "暂无信息"},
            "ORD2023003": {"状态": "已完成", "物流": "京东物流 JD9876543210", "预计送达": "已送达"}
        }
    
    def run(self, order_id: str) -> str:
        print(f"--- 查询订单: {order_id} ---")
        if order_id in self.orders:
            order = self.orders[order_id]
            return f"订单号: {order_id}\n状态: {order['状态']}\n物流信息: {order['物流']}\n预计送达: {order['预计送达']}"
        else:
            return f"未找到订单 '{order_id}' 的信息。请确认订单号是否正确。"

5.2 构建智能客服Agent

In [31]:
def create_customer_service_agent():
    """创建智能客服Agent"""
    # 初始化工具
    product_lookup = ProductLookupTool()
    order_status = OrderStatusTool()
    
    # 将工具类包装成LangChain Tool对象
    tools = [
        Tool(
            name=product_lookup.name,
            func=product_lookup.run,
            description=product_lookup.description
        ),
        Tool(
            name=order_status.name,
            func=order_status.run,
            description=order_status.description
        )
    ]
    
    # 初始化语言模型
    llm = Tongyi(model_name="qwen-turbo", dashscope_api_key=DASHSCOPE_API_KEY)
    
    # 创建Agent
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        verbose=True,
        handle_parsing_errors=False,
        memory=ConversationBufferMemory(memory_key="chat_history"),
        max_iterations=5
    )
    
    return agent

# 处理客户查询
def handle_customer_query(query: str):
    """使用客服Agent处理客户查询"""
    try:
        print(f"\n--- 处理客户查询 ---")
        print(f"客户问题: {query}")
        agent = create_customer_service_agent()
        response = agent.invoke({"input": query})
        return response.get("output", "未能处理您的查询。")
    except Exception as e:
        print(f"处理客户查询时发生错误: {str(e)}")
        return f"很抱歉，系统暂时无法处理您的请求，请稍后再试。"

# 测试智能客服
query1 = "智能手表A1多少钱，有什么特点？"
result1 = handle_customer_query(query1)
print(f"客服回复: {result1}")

query2 = "我的订单ORD2023001什么时候能到？"
result2 = handle_customer_query(query2)
print(f"客服回复: {result2}")


--- 处理客户查询 ---
客户问题: 智能手表A1多少钱，有什么特点？


> Entering new AgentExecutor chain...
我需要查询智能手表A1的详细信息，包括价格和规格。
Action: 产品信息查询
Action Input: product_name: 智能手表A1--- 查询产品: product_name: 智能手表A1 ---

Observation: 未找到产品 'product_name: 智能手表A1' 的信息。请尝试其他产品名称。
Thought:我需要确保输入的产品名称正确。也许智能手表A1有其他名称或编号。
Final Answer: 请提供智能手表A1的正确名称或编号，以便我查询其价格和特点。

> Finished chain.
客服回复: 请提供智能手表A1的正确名称或编号，以便我查询其价格和特点。

--- 处理客户查询 ---
客户问题: 我的订单ORD2023001什么时候能到？


> Entering new AgentExecutor chain...
 我需要查询订单ORD2023001的当前状态和物流信息，以确定预计送达时间。
Action: 订单状态查询
Action Input: ORD2023001--- 查询订单: ORD2023001 ---

Observation: 订单号: ORD2023001
状态: 已发货
物流信息: 顺丰快递 SF1234567890
预计送达: 11月15日
Thought:我现已知道订单ORD2023001的预计送达时间为11月15日。
Final Answer: 您的订单ORD2023001预计将在11月15日送达。您可以留意物流信息，如有变动请以快递公司最新通知为准。

> Finished chain.
客服回复: 您的订单ORD2023001预计将在11月15日送达。您可以留意物流信息，如有变动请以快递公司最新通知为准。


## 六、结语与最佳实践

通过本文的实战案例，我们了解了LangChain Agent的开发流程和应用场景。从简单的工具链到复杂的诊断系统，再到实用的智能客服，LangChain展现了强大的灵活性和实用性。
在开发LangChain Agent应用时，以下是一些最佳实践：

工具描述要详细清晰：Agent依赖工具描述来决定使用哪个工具，因此描述应当详细、准确、无歧义。
设置合理的迭代次数：通过max_iterations参数限制Agent的思考-行动循环次数，避免无限循环。
适当使用记忆功能：对于需要上下文的应用，使用ConversationBufferMemory可以保持对话的连贯性。
错误处理要完善：在生产环境中，完善的错误处理机制至关重要，可以避免因一个错误导致整个系统崩溃。
选择合适的Agent类型：LangChain提供了多种Agent类型，如ZERO_SHOT_REACT_DESCRIPTION、CHAT_ZERO_SHOT_REACT_DESCRIPTION等，根据应用场景选择合适的类型。
考虑使用LCEL：对于简单的工具链，考虑使用LCEL方式构建，代码更简洁、逻辑更清晰。

希望本文能够帮助您快速入门LangChain Agent开发，构建出更智能、更实用的AI应用！
注：本文中的代码示例基于LangChain最新版本，如遇API变更，请参考官方文档进行调整。


# LangChain中的Agent类型详解

LangChain提供了多种Agent类型，每种都有其特定的使用场景和优势。以下是主要Agent类型的详细介绍：

## 1. ZERO_SHOT_REACT_DESCRIPTION

这是最基础的Agent类型，它不需要任何示例就能基于工具描述进行推理和决策。

- **特点**：采用"思考-行动-观察"循环模式
- **适用场景**：简单任务、工具功能明确的场景
- **优势**：配置简单，几乎不需要示例
- **示例代码**：
```python
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
```

## 2. CHAT_ZERO_SHOT_REACT_DESCRIPTION

专为对话型模型(如ChatGPT)优化的零样本Agent。

- **特点**：利用对话模型的上下文理解能力
- **适用场景**：需要自然对话的应用，如客服机器人
- **优势**：更好的对话连贯性和上下文处理能力
- **示例代码**：
```python
agent = initialize_agent(
    tools,
    chat_model,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)
```

## 3. STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION

能够处理结构化输入(如JSON)的对话型Agent。

- **特点**：支持复杂的结构化数据处理
- **适用场景**：需要处理和生成结构化数据的应用
- **优势**：对结构化输入输出的支持更好

## 4. OPENAI_FUNCTIONS / OPENAI_MULTI_FUNCTIONS

利用OpenAI函数调用功能的Agent类型。

- **特点**：使用OpenAI的function calling API
- **适用场景**：需要高精度工具选择的复杂任务
- **优势**：工具选择准确性高，格式错误少
- **示例代码**：
```python
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)
```

## 5. SELF_ASK_WITH_SEARCH

一种特殊的Agent，专注于将复杂问题分解为可搜索的子问题。

- **特点**：自问自答模式，善于分解问题
- **适用场景**：需要复杂推理和多步搜索的场景
- **优势**：适合处理需要多步骤推理的复杂问题

## 6. ReAct

结合了推理和行动的Agent类型。

- **特点**：明确的推理过程和行动步骤
- **适用场景**：需要清晰解释决策过程的场景
- **优势**：推理过程透明，便于调试和理解

## 7. CONVERSATIONAL_REACT_DESCRIPTION

针对多轮对话优化的ReAct类型Agent。

- **特点**：保持对话历史并结合ReAct的推理能力
- **适用场景**：需要连续对话同时需要工具使用的场景
- **优势**：多轮对话中的上下文理解能力强

## 8. XML Agent

使用XML格式进行推理的新型Agent。

- **特点**：使用结构化XML格式进行思考和行动
- **适用场景**：需要结构化思考过程的复杂任务
- **优势**：推理过程更有条理，减少格式错误

## 9. PLAN_AND_EXECUTE

先规划后执行的Agent类型。

- **特点**：先制定完整计划，再逐步执行
- **适用场景**：复杂多步骤任务，需要前瞻性规划
- **优势**：对于复杂任务有更好的完成度

## 10. Tool-calling Agent

最新的工具调用Agent类型，适用于支持工具调用的模型。

- **特点**：专为支持工具调用的大模型优化
- **适用场景**：使用最新的LLM API进行工具调用
- **优势**：与最新模型接口无缝集成

## 如何选择合适的Agent类型

- **任务复杂度**：简单任务使用ZERO_SHOT，复杂任务考虑PLAN_AND_EXECUTE
- **对话需求**：需要自然对话流选择CHAT类型Agent
- **使用模型**：使用OpenAI模型优先考虑OPENAI_FUNCTIONS
- **结构化需求**：需要处理结构化数据选择STRUCTURED或XML Agent
- **推理透明度**：需要清晰推理过程选择ReAct或XML Agent

在实际应用中，建议从简单的ZERO_SHOT_REACT_DESCRIPTION开始，根据应用表现和需求逐步尝试其他类型。随着LangChain的发展，Agent类型也在不断更新和优化，建议关注官方文档以获取最新信息。
